# Common

In [5]:
# グローバル変数
node_id_dict = {} # key: userId, value: nodeId
node_id_cnt = 1

# エッジリストファイルから読み込んでをセット型の変数に格納
# 各エッジはタプルとして扱う
# リストだと相互フォロー探索がO(n^2)になってしまう
def get_edgelist_from_file(filename):
    edgelist = set()
    file = open(filename, 'r')
    
    for line in file:
        line = line.rstrip('\n') # 改行処理
        tmp = line.split() # 1行をスペースで分割してリストに格納
        edge = tuple(tmp) # リストをタプルに変換
        edgelist.add(edge)
        
    file.close()
    
    return edgelist

# エッジリストをファイルに出力
def edgelist_to_file(edgelist, filename):
    file = open(filename, 'w')
    
    for edge in edgelist:
        tmp = edge[0] + ' ' + edge[1] + '\n'
        file.write(tmp)
        
    file.close()
    
# node_id_dictとnode_id_cntの初期化
def init_node_id_dict():
    global node_id_dict
    global node_id_cnt

    node_id_dict = {}
    node_id_cnt = 1

# 1オリジンのnodeIdを返す
# nodeIdは文字列で返す
def node_id_to_oneorigin(node_id_from):
    global node_id_dict
    global node_id_cnt

    if node_id_from in node_id_dict:
        return str(node_id_dict[node_id_from])
    else:
        node_id_dict[node_id_from] = node_id_cnt
        node_id_cnt += 1
        return str(node_id_to_oneorigin(node_id_from))

# エッジリストに1オリジンのnodeIdを振る
# 最後に対応表をファイルに出力
# (用途1)userIdをnodeIdに変換
# (用途2)コミュニティのエッジリストを1オリジンに
# エッジリストのuserIdをnodeIdに変換
def edgelist_to_oneorigin(edgelist):
    # グローバル変数の初期化
    init_node_id_dict()
    
    oneorigin_edgelist = set()
    
    for edge in edgelist:
        tmp = (node_id_to_oneorigin(edge[0]), node_id_to_oneorigin(edge[1]))
        oneorigin_edgelist.add(tmp)
        
    file = open(node_id_dict_file, 'w')
    for user_id, node_id in node_id_dict.items():
        tmp = str(user_id) + ' ' + str(node_id) + '\n'
        file.write(tmp)
        
    file.close()
    
    return oneorigin_edgelist

# コミュニティファイルを読み込んで辞書型変数に格納
# key:community_id
# value:node_idのリスト
# 整数は文字列として扱う
def get_community_from_file():
    community_dic = {}
    file = open(community_file, 'r')
    
    for line in file:
        line = line.rstrip('\n') # 改行処理
        line = line.split(';') # 1行 ; で分割してリストに格納
        
        node_id = str(line[0].replace('"', '')) # "" を削除
        community_id = str(line[1])
        
        if community_id in community_dic:
            # このコミュニティのリストがすでに作られているとき
            community_dic[community_id].append(node_id)
        else:
            # このコミュニティの初めてのnodeのとき
            community_dic[community_id] = [node_id]
        
    file.close()
    
    return community_dic

# Define

In [6]:
# ファイル名
# 本番データ
# エッジリスト加工前
edgelist_file_from = "../vent_data/vent.edgelist"
# エッジリスト加工後
edgelist_file_to = "../vent_data/new_vent.edgelist"
# コミュニティデータ
community_file = "../vent_data/community.txt"
# id対応表
node_id_dict_file = "../vent_data/nodeid_dict.txt"
# 感情
emotions_file = "../vent_data/emotions.csv" 
new_emotions_file = "../vent_data/new_emotions.txt" # 前処理後
# メタデータ
metadata_file = "../vent_data/vents_metadata.csv"
new_metadata_file = "../vent_data/new_vents_metadata.txt" # 感情による前処理後
community_metadata_file = "../vent_data/metadata{}.txt" # ユーザによる前処理後

# テストデータ
# エッジリスト加工前
#edgelist_file_from = "../vent_data/mini_vent.edgelist"
# エッジリスト加工後
#edgelist_file_to = "../vent_data/mini_new_vent.edgelist"
# コミュニティデータ
#community_file = "../vent_data/mini_community.txt"

# 観察対象の感情カテゴリ(セット型)
emotion_categories = {
    '0af046e4-bf8d-4776-8d05-2e5128568330', # Anger
    '2a1e8661-ff21-4dae-971e-587438ba35ea', # Sadness
    '2f7d9ab1-c412-4734-9545-84b3cad01e59', # Fear
    '7db21f2c-f0c8-48e1-8ed3-099124cb5c08', # Surprise
    '79af7aa9-660e-49fc-9576-323b6a503179', # Happiness
    '4fdc3a9c-48bc-4d8d-b01e-0d6e3097307c', # Positivity
}
# マッピング用(辞書型)
emotion_categories_map = {
    '0af046e4-bf8d-4776-8d05-2e5128568330': 'Anger',
    '2a1e8661-ff21-4dae-971e-587438ba35ea': 'Sadness',
    '2f7d9ab1-c412-4734-9545-84b3cad01e59': 'Fear',
    '7db21f2c-f0c8-48e1-8ed3-099124cb5c08': 'Surprise',
    '79af7aa9-660e-49fc-9576-323b6a503179': 'Happiness',
    '4fdc3a9c-48bc-4d8d-b01e-0d6e3097307c': 'Positivity'
}

# 観察対象の期間
survey_time_list = [
    ('2017-1-1',  '2017-1-31',  'January'),
    ('2017-3-1',  '2017-3-31',  'March'),
    ('2017-5-1',  '2017-5-31',  'May'),
    ('2017-7-1',  '2017-7-31',  'July'),
    ('2017-9-1',  '2017-9-30',  'September'),
    ('2017-11-1', '2017-11-30', 'November')
]

# 観察対象全期間
survey_time_all = ('2017-1-1',  '2017-11-30')

# 観察対象のコミュニティ
survey_communities = [
    'all',
    '8',
    '11',
    '18',
    '21',
    '24',
    '26',
    '28',
    '35',
    '115'
]

# エッジリストデータの加工
- オープンデータのidを1オリジンのidに対応付ける
- グラフが無向グラフになるようにする(相互フォローのみにエッジを張る)
- エッジリストはソースの中ではリストで扱うようにする

In [7]:
# グラフの無向化
# 相互フォローのみにエッジを張る
def to_undirected(edgelist):
    undirected_edgelist = set()
    
    for tpl in edgelist:
        # 逆向きのエッジが存在したら
        if (tpl[1], tpl[0]) in edgelist:
            undirected_edgelist.add(tpl)
    
    return undirected_edgelist
    
# main-エッジリストを加工してファイル出力
def create_modified_edgelist():
    print('create_modified_edgelist is started!')
    
    # ファイルからリストに変換
    edgelist = get_edgelist_from_file(edgelist_file_from)
    print('get_edgelist_from_file() is completed!')
    
    # グラフの無向化
    edgelist = to_undirected(edgelist)
    print('to_undirected() completed!')
    
    # userIdをnodeIdに変換
    edgelist = edgelist_to_oneorigin(edgelist)
    print('edgelist_to_oneorigin() is completed!')
    
    # ファイル出力
    edgelist_to_file(edgelist, edgelist_file_to)
    print('edgelist_to_file() is completed!')
    print('All completed! {} edges are created.'.format(len(edgelist)))

In [37]:
create_modified_edgelist()

create_modified_edgelist is started!
get_edgelist_from_file() is completed!
to_undirected() completed!
edgelist_to_oneorigin() is completed!
edgelist_to_file() is completed!
All completed! 7248521 edges are created.


# コミュニティネットワークを検証
- Rでコミュニティ抽出→ファイル出力
- コミュニティファイルの前処理
    - 拡張子を.csv -> .txt にする
    - 一行目を削除する

In [8]:
# main-コミュニティ分割の結果を検証する
def check_community():
    print('check_community() is started!')
    
    # 一度にノード数を要求するとメモリがあふれるので数コミュニティ指定する
    community_id_from = int(input("imput a communityId_from"))
    community_id_to = int(input("imput a communityId_to"))
    
    # ファイルからコミュニティデータを取得
    community = get_community_from_file()
    print('get_community_from_file() is completed!')
    
    # 指定したコミュニティに含まれるノードの数を求め出力
    for community_id in range(community_id_from, community_id_to):
        community_id = str(community_id)
        node_num = len(community[community_id])
        print('community{} has {} nodes.'.format(community_id, node_num))
        
# main-ノード数が200~999のコミュニティを出力する
def get_terget_community():
    print('get_terget_community() is started!')
    
    # 一度にノード数を要求するとメモリがあふれるので数コミュニティ指定する
    community_id_from = int(input("imput a communityId_from"))
    community_id_to = int(input("imput a communityId_to"))
    
    # ファイルからコミュニティデータを取得
    community = get_community_from_file(community_file)
    print('get_community_from_file() is completed!')
    
    # 指定したコミュニティに含まれるノードの数を求め出力
    for community_id in range(community_id_from, community_id_to):
        community_id = str(community_id)
        node_num = len(community[community_id])
        if 200 <= node_num and node_num < 1000:
            print('community{} : {} nodes'.format(community_id, node_num))

In [11]:
get_terget_community()

get_terget_community() is started!
imput a communityId_from1
imput a communityId_to200
get_community_from_file() is completed!
community8 : 603 nodes
community11 : 248 nodes
community18 : 250 nodes
community21 : 582 nodes
community24 : 479 nodes
community26 : 253 nodes
community28 : 378 nodes
community35 : 351 nodes
community115 : 534 nodes


In [39]:
check_community()

check_community() is started!
imput a communityId_from1
imput a communityId_to100
get_community_from_file() is completed!
community1 has 1 nodes.
community2 has 128459 nodes.
community3 has 114885 nodes.
community4 has 139775 nodes.
community5 has 114728 nodes.
community6 has 47052 nodes.
community7 has 8574 nodes.
community8 has 603 nodes.
community9 has 1448 nodes.
community10 has 3789 nodes.
community11 has 248 nodes.
community12 has 1584 nodes.
community13 has 13 nodes.
community14 has 63 nodes.
community15 has 5 nodes.
community16 has 35 nodes.
community17 has 61 nodes.
community18 has 250 nodes.
community19 has 3 nodes.
community20 has 2 nodes.
community21 has 582 nodes.
community22 has 153 nodes.
community23 has 26 nodes.
community24 has 479 nodes.
community25 has 4 nodes.
community26 has 253 nodes.
community27 has 92 nodes.
community28 has 378 nodes.
community29 has 74 nodes.
community30 has 142 nodes.
community31 has 15 nodes.
community32 has 2 nodes.
community33 has 6 nodes.


# データの前処理
- emotion_category.csv
    - いらないカテゴリは消しておく
- emotion.csv
    - いらないカテゴリに属する感情は消しておく
- vent_metadata.csv
    - いらないユーザの行は消しておく
    - いらない感情の行は消しておく
    - いらない時間の行は消しておく

In [9]:
# emotions.csvの前処理
# new_emotions.txtへ出力
# 絵文字がうざいためemotion_idとemotion_category_idのみ
def preprocessing_emotions():
    file = open(emotions_file, 'r', encoding = 'utf-8')
    new_file = open(new_emotions_file, 'w', encoding = 'utf-8')
    
    for line in file:
        line = line.rstrip('\n') # 改行処理
        emotion = line.split(',') # 1行をスペースで分割してリストに格納
        
        if emotion[1] in emotion_categories: # 前処理
            tmp = emotion[0] + ' ' + emotion[1] + '\n'
            new_file.write(tmp)
        
    file.close()
    new_file.close()
    
    print('preprocessing_emotions() is completed!')

# new_emotions.txtを読み込んで観察対象の感情をセット型変数に格納
# emotion_idのみが格納される
def get_emotions_from_file():
    emotions = set()
    file = open(new_emotions_file, 'r')
    
    for line in file:
        line = line.rstrip('\n') # 改行処理
        tmp = line.split() # 1行をスペースで分割してリストに格納
        emotions.add(tmp[0]) # emotion_idのみ格納
        
    file.close()
    
    return emotions

# vents_metadata.csvの前処理
# 感情によるデータ圧縮
# 観察対象の感情のみ残す
# reactionもいらないので消す
def preprocessing_metadata_by_emotions():
    # 観察対象の感情
    emotions = get_emotions_from_file()
    
    file = open(metadata_file, 'r', encoding = 'utf-8')
    new_file = open(new_metadata_file, 'w', encoding = 'utf-8')
    
    for line in file:
        line = line.rstrip('\n') # 改行処理
        metadata = line.split(',') # 1行を,で分割してリストに格納
        
        if metadata[0] in emotions: # 前処理
            tmp = metadata[0] + ' ' + metadata[1] + ' ' + metadata[2] + '\n'
            new_file.write(tmp)
        
    file.close()
    new_file.close()
    
    print('preprocessing_metadata_by_emotions() is completed!')
    
# nodeIdをuserIdに変換するための辞書を取得
# key: nodeId, value: userId で返す
# nodeId->userIdの変換で使いたいので、逆にしておく
def get_node_id_dict_from_file():
    node_id_dict = {}
    
    file = open(node_id_dict_file, 'r')
    
    for line in file:
        line = line.rstrip('\n') # 改行処理
        tmp = line.split() # 1行をスペースで分割してリストに格納
        node_id_dict[tmp[1]] = tmp[0]
        
    return node_id_dict
    
# 特定コミュニティに含まれるユーザ一覧(set型)を取得
# nodeIdではなくuserIdで取得
def get_users_in_community(community_id):
    # 辞書取得
    node_id_dict = get_node_id_dict_from_file()
    
    # 該当ユーザ取得
    communitys = get_community_from_file()
    community = communitys[community_id]
    
    users = set()
    for user in community:
        # nodeIdではなくuserIdを格納
        users.add(node_id_dict[user])
        
    return users

# vents_metadata.csvの前処理
# ユーザによるデータ圧縮
# 観察対象のユーザのみ残す
def preprocessing_metadata_by_users():
    community_id = input('communityId : ')
    # 出力ファイルのファイル名生成
    filename = community_metadata_file.format(community_id)
    # 観察対象のユーザを取得
    users = get_users_in_community(community_id)
    
    file = open(new_metadata_file, 'r')
    new_file = open(filename, 'w')
    
    for line in file:
        line = line.rstrip('\n') # 改行処理
        metadata = line.split() # 1行をスペースで分割してリストに格納
        
        if metadata[1] in users: # 前処理
            tmp = metadata[0] + ' ' + metadata[1] + ' ' + metadata[2] + '\n'
            new_file.write(tmp)
        
    file.close()
    new_file.close()
    
    print('preprocessing_metadata_by_users() is completed!')
        
# 時間データの形式変換
# yyyy-mm-dd -> [yyyy, mm, dd]
# time : string
# return : [int, int, int]
def convert_time(time):
    tmp = time.split('-')
    re = []
    for i in tmp:
        re.append(int(i))
        
    return re

# 指定した時間が観察対象内かどうかを判定する
# コンバート済みのtimeを引数として与えることに注意
# time, time_from, time_to: list
# return: boolen
def in_time(time, time_from, time_to):
    return (time_from[0] <= time[0] and time[0] <= time_to[0]) and (time_from[1] <= time[1] and time[1] <= time_to[1]) and (time_from[2] <= time[2] and time[2] <= time_to[2]) 
     
# 出力ファイル名を生成
# ex: 8_170901_170930.txt
def get_metadata_filename(survey_time_from, survey_time_to, community_id, backup = False):
    if backup:
        dirname = '../vent_data/buckup/'
    else:
        dirname = '../vent_data/preprocessed/'
    
    time_from = convert_time(survey_time_from)
    time_to = convert_time(survey_time_to)
    filename =  community_id + '_' + str(time_from[0] - 2000) + str(time_from[1]).zfill(2) + str(time_from[2]).zfill(2) + '_' + str(time_to[0] - 2000) + str(time_to[1]).zfill(2) + str(time_to[2]).zfill(2) + '.txt'
    
    return dirname + filename
    
# vents_metadata.csvの前処理
# 時間によるデータ圧縮
# community_idに"all"と入力するとユーザによる前処理がされていないデータに対して前処理をかけることができる
def preprocessing_metadata_by_time(survey_time_from, survey_time_to, community_id):
    # 観察対象期間
    time_from = convert_time(survey_time_from)
    time_to = convert_time(survey_time_to)
    
    # 観察対象コミュニティ
    if community_id == '':
        # 個別実験したいとき用
        community_id = input('community_id : ')
    if community_id == 'all':
        filename = new_metadata_file
    else:
        filename = community_metadata_file.format(community_id)
    file = open(filename, 'r')
    
    new_filename = get_metadata_filename(survey_time_from, survey_time_to, community_id)
    new_file = open(new_filename, 'w')
    
    for line in file:
        line = line.rstrip('\n') # 改行処理
        metadata = line.split() # 1行をスペースで分割してリストに格納
        
        time = convert_time(metadata[2])
        if in_time(time, time_from, time_to): # 前処理
            tmp = metadata[0] + ' ' + metadata[1] + ' ' + metadata[2] + '\n'
            new_file.write(tmp)
            
    file.close()
    new_file.close()
    
    print('preprocessing_metadata_by_time() is completed!')
    
# コミュニティ内でのユーザのアクティビティを検証する
# TODO : 間違えて先にカテゴリを付与してしまうと動かない(metadata[1]がemotion_category_idになってしまうため)
def show_activity_of_community(survey_time_from, survey_time_to, community_id):
    filename = get_metadata_filename(survey_time_from, survey_time_to, community_id)
    file = open(filename, 'r')
    
    # 感情発信回数のカウント
    total = 0
    user_num = {}
    for line in file:
        line = line.rstrip('\n') # 改行処理
        metadata = line.split() # 1行をスペースで分割してリストに格納
        user = metadata[1]
        
        total += 1
        if user in user_num:
            user_num[user] += 1 
        else:
            user_num[user] = 1
            
    print('node : {}'.format(len(user_num)))
    print('emotion : {}'.format(total))
        
    # アクティビティの計算
    acutivities = {}
    for user, num in user_num.items():
        acutivities[user] = round((num / total) * 100, 2) # 百分率, 小数点以下切り捨て
        
    # 結果出力
    # 上位100ユーザのみ表示させてみる
    max_print = 100
    print_num = 0
    # valueのソート
    # acutivities_sortedは(key, value)のタプルを要素とするリストになるので注意
    acutivities_sorted = sorted(acutivities.items(), key=lambda x:x[1], reverse=True)
    for item in acutivities_sorted:
        user = item[0]
        activity = item[1]
        if print_num > max_print:
            break
        print('{} : {} %'.format(user, activity))
        print_num += 1
        
# new_emotions.txtを読み込んで辞書型変数に格納
# key:emotion_id, value:emotion_category_id
def get_emotion_category_dict_from_file():
    file = open(new_emotions_file, 'r')
    
    emotion_category_dict = {}
    for line in file:
        line = line.rstrip('\n') # 改行処理
        tmp = line.split() # 1行をスペースで分割してリストに格納
        emotion_category_dict[tmp[0]] = tmp[1]
        
    file.close()
        
    return emotion_category_dict

# 前処理済みメタデータ(n_yymmdd_yymmdd.txt)にemotion_category_idを付与
# TODO : 感情の数え上げのほうに入れる？
def add_emotion_category_id_to_metadata(survey_time_from, survey_time_to, community_id):
    # emotion_category_dictの取得
    emotion_category_dict = get_emotion_category_dict_from_file()
    
    # emotion_category_idを付与の付与
    filename = get_metadata_filename(survey_time_from, survey_time_to, community_id)
    file = open(filename, 'r')
    tmp_file = open('../vent_data/tmp.txt', 'w')
    for line in file:
        line = line.rstrip('\n') # 改行処理
        metadata = line.split() # 1行をスペースで分割してリストに格納
        
        # 2回以上実行してしまうとデータがぶっ壊れてしまうので回避する
        if len(metadata) >= 4:
            return print('You have already added emotion_category_id to this metadata.')
        
        tmp = metadata[0] + ' ' + emotion_category_dict[metadata[0]] + ' ' + metadata[1] + ' ' + metadata[2] + '\n'
        tmp_file.write(tmp)
        
    file.close()
    tmp_file.close()
    
    # 同じファイル名に上書き
    # 同じファイル名に上書きしたいのでバックアップを取る
    import os
    import shutil
    src = '../vent_data/tmp.txt'
    origin = filename
    buckup = get_metadata_filename(survey_time_from, survey_time_to, community_id, True)
    # バックアップ
    shutil.copyfile(origin, buckup)
    # 上書き
    shutil.copyfile(src, origin)
    # tmpファイルの削除
    os.remove(src)
    
    print('add_emotion_category_id_to_metadata() is completed!')

In [7]:
# emotions.csvの前処理
preprocessing_emotions()

preprocessing_emotions() is completed!


In [8]:
# 感情によるメタデータの前処理
preprocessing_metadata_by_emotions()

preprocessing_metadata_by_emotions() is completed!


In [9]:
# ユーザ(コミュニティ)によるメタデータの前処理
preprocessing_metadata_by_users()

communityId : 8
preprocessing_metadata_by_users() is completed!


# ここからやればok

In [11]:
# 観察期間によるメタデータの前処理
preprocessing_metadata_by_time()

community_id : 8
preprocessing_metadata_by_time() is completed!


In [12]:
# コミュニティのアクティビティの検証
show_activity_of_community('8_170101_170131.txt')

node : 51
emotion : 391
377165f1-a9e9-4922-ae3e-2f75cfa83c5f : 23.02 %
cd21e79f-f435-458d-8898-764a8c8afcf8 : 6.65 %
41264d6b-1041-3900-8b6f-6423b578f3e9 : 5.37 %
eae2efb2-b915-4465-a0e6-4d50120008a1 : 4.86 %
6da4695d-06da-4000-abe8-2c779ba8777c : 4.6 %
f114fa2e-dfc3-4add-8c0b-0db1c811fa40 : 4.6 %
c1751066-e77c-45ff-b17b-daee7d327003 : 3.84 %
ae102f7a-474e-40ad-92a9-4373fe7b252c : 3.32 %
ef928cbd-c427-462a-a9ce-bb122cc772ed : 3.07 %
8f778292-ff20-3111-95f3-2a0926f448c5 : 2.81 %
024e0547-da18-4590-b9a7-65ad101d7ae7 : 2.81 %
8680e662-db12-4d9b-b1ef-482af8a73068 : 2.56 %
643df8aa-8b9f-441e-a895-b9de313bc026 : 2.56 %
478ba9d4-db87-47e5-9c32-0773f0660335 : 2.3 %
8d112bd6-f053-496f-b51a-23d29bf24c40 : 2.05 %
cd951ffc-2308-412d-b09f-1938b352c821 : 1.79 %
231ee94f-f098-4441-8e57-1224bfdc389b : 1.53 %
75ff6ced-4c2c-3d22-b35f-bfbf9c87413a : 1.53 %
e94edfae-cf15-4c52-9f82-f677bf27043b : 1.28 %
bb270f17-0602-44b9-abed-67cd2016d2b7 : 1.28 %
48eacc97-05a2-4755-aa57-7e5d26c271df : 1.28 %
8f908092-8e5

In [13]:
# メタデータに感情カテゴリを付与する
add_emotion_category_id_to_metadata('8_170101_170131.txt')

add_emotion_category_id_to_metadata() is completed!


# 感情カテゴリの数え上げ

In [10]:
def show_emotion_category_rate(survey_time_from, survey_time_to, community_id):
    # 相対パスを生成
    filename = get_metadata_filename(survey_time_from, survey_time_to, community_id)
    file = open(filename, 'r')
    
    # 感情カテゴリのカウント
    total = 0
    emotion_category_num = {}
    for emotion_category_id in emotion_categories:
        emotion_category_num[emotion_category_id] = 0
    
    for line in file:
        line = line.rstrip('\n') # 改行処理
        metadata = line.split() # 1行をスペースで分割してリストに格納
        emotion_category_id = metadata[1]
        
        total += 1
        emotion_category_num[emotion_category_id] += 1
            
    print('total : {} emotions'.format(total))
        
    # カテゴリ割合の計算
    emotion_category_rate = {}
    for emotion_category_id in emotion_categories:
        emotion_category_rate[emotion_category_id] = 0
        
    for emotion_category_id, num in emotion_category_num.items():
        emotion_category_rate[emotion_category_id] = round((num / total) * 100, 2) # 百分率, 小数点以下切り捨て
        
    # 結果出力
    for emotion_category_id, category in emotion_categories_map.items():
        rate = emotion_category_rate[emotion_category_id]
        print('{} : {} %'.format(category, rate))

In [19]:
show_emotion_category_rate('8_170101_170131.txt')

total : 391 emotions
Anger : 21.99 %
Sadness : 24.04 %
Fear : 11.0 %
Surprise : 6.91 %
Happiness : 26.85 %
Positivity : 9.21 %


# 一連の計算処理をまとめってやる

In [11]:
# 各期間・各コミュニティ
def show_monthly_result():
    for survey_time in survey_time_list:
        survey_time_from = survey_time[0]
        survey_time_to   = survey_time[1]
        print(survey_time[2] + ' :')
        print()
        
        for community_id in survey_communities:
            print('[' + community_id + '] -------------------------------------------------')
        
            # 観察期間によるメタデータの前処理
            preprocessing_metadata_by_time(survey_time_from, survey_time_to, community_id)
            print()

            # コミュニティのアクティビティの検証
            show_activity_of_community(survey_time_from, survey_time_to, community_id)
            print()

            # メタデータに感情カテゴリを付与する
            add_emotion_category_id_to_metadata(survey_time_from, survey_time_to, community_id)
            print()

            # 感情の分布を検証
            show_emotion_category_rate(survey_time_from, survey_time_to, community_id)
            print()
            
        print('=======================================================')
        
# 全期間・各コミュニティ
def show_alltime_result():
    survey_time_from = survey_time_all[0]
    survey_time_to   = survey_time_all[1]
    
    for community_id in survey_communities:
        print('[' + community_id + '] -------------------------------------------------')

        # 観察期間によるメタデータの前処理
        preprocessing_metadata_by_time(survey_time_from, survey_time_to, community_id)
        print()

        # コミュニティのアクティビティの検証
        show_activity_of_community(survey_time_from, survey_time_to, community_id)
        print()

        # メタデータに感情カテゴリを付与する
        add_emotion_category_id_to_metadata(survey_time_from, survey_time_to, community_id)
        print()

        # 感情の分布を検証
        show_emotion_category_rate(survey_time_from, survey_time_to, community_id)
        print()

In [42]:
show_monthly_result()

January :

[all] -------------------------------------------------
preprocessing_metadata_by_time() is completed!

node : 43663
emotion : 429065
174d8703-e501-4c1c-88eb-25ba49a8b4b1 : 0.29 %
559ca24a-96f9-40b3-b924-eb62c0960ff0 : 0.23 %
3941fdbd-3679-46a0-92cf-6abfe26c8bae : 0.21 %
9f5a5c52-ce62-4808-b354-069a822ac646 : 0.15 %
195db094-0bf8-4b8a-8cda-2a0b7323723e : 0.14 %
db35fb2e-98bb-4753-be1c-dfae5f64f9e1 : 0.13 %
0be36fd6-3d5f-495a-b53b-039500394466 : 0.13 %
71059cde-30e1-4e72-9557-4fd76f641739 : 0.12 %
31670e18-1fe6-33f3-8586-cec54d23facf : 0.11 %
bfb2d368-74cb-41eb-99f4-605b93ad252b : 0.11 %
bc840ba0-546e-4b05-b738-04021d58d8c8 : 0.1 %
41488791-aa76-4c37-8a48-530adf0cce35 : 0.1 %
589d4f90-fc93-42a4-a605-eff0c3f2aa0f : 0.1 %
47371321-5721-385a-8c20-cfc48c00f1c4 : 0.09 %
8405290a-b308-4df2-a9d3-10205d73643f : 0.09 %
bbe41daf-7927-464c-8adb-e046f6492f6e : 0.09 %
5c6c9980-39e6-4519-aa00-2ceb10606e0a : 0.09 %
eb7fc33f-d21f-40ab-8f8c-31768c748b2e : 0.09 %
a1ea20f5-e2ed-4f73-90fa-041a14

node : 14
emotion : 107
fda0171e-f5e5-3dcc-b69f-a54bf8269fca : 29.91 %
466a5cc2-7664-4916-9b94-a7a6b2fca37a : 22.43 %
d7ecfbb0-2e61-4659-8594-e4c856c7c50d : 8.41 %
481780a7-0762-46a6-b2f5-d6677307ac7c : 7.48 %
c34c40fa-c742-3a9c-85bb-35ef49811a41 : 6.54 %
93d751f6-d03a-426b-91c5-9ed8ae0046cc : 4.67 %
24068499-1301-3e4c-b81d-4af417c35bbc : 3.74 %
820ceb12-0b29-41af-801d-1e155ea1f440 : 3.74 %
e2a007da-265f-4f57-902d-c49a74dba34e : 2.8 %
dc5c4c6c-68eb-4342-bcf7-f38dedb9eb40 : 2.8 %
63520e83-3f4f-3b08-b1f9-641a5bd6bc12 : 2.8 %
64cf9677-b0b3-3c95-a5f5-30eadd8d5564 : 1.87 %
80a7550c-1e82-433f-a605-6c13ab38fcf7 : 1.87 %
24a74260-511f-4d4c-aa5b-00459a972d98 : 0.93 %

add_emotion_category_id_to_metadata() is completed!

total : 107 emotions
Anger : 14.95 %
Sadness : 25.23 %
Fear : 15.89 %
Surprise : 13.08 %
Happiness : 19.63 %
Positivity : 11.21 %

[21] -------------------------------------------------
preprocessing_metadata_by_time() is completed!

node : 35
emotion : 352
d68c1780-f18c-4e47-bc

preprocessing_metadata_by_time() is completed!

node : 25
emotion : 122
a64695bb-f2f3-4585-a1e9-04eb8a8cc479 : 15.57 %
ee5340a2-1b94-4b10-a327-b3e450d44616 : 13.11 %
0f41fc9a-b9b4-4315-9fa2-f8a3492ce18a : 10.66 %
57a2abc3-f0e6-367d-832a-75fa4d1a908e : 9.84 %
ef27ad8a-11ed-4204-98db-2f352a7c526f : 6.56 %
6a848c8c-2a56-39f0-9cdd-5463b28e2c4c : 5.74 %
d635dfbd-c01a-4cec-9530-d21609a77be1 : 4.92 %
b5771947-5e01-4176-ad41-871a62025a90 : 4.92 %
45dc4b1e-c7c7-4883-a185-59d43d3b0cd4 : 4.92 %
e76c80b6-7482-4070-a1e3-99808f26755b : 3.28 %
34573907-4a54-3ac3-b459-dc998f0c6668 : 3.28 %
af5703d3-d2b7-4d62-a0eb-f0936cb21449 : 3.28 %
1a5780c8-1891-4ccc-b78b-b9d145a883d7 : 1.64 %
20029adc-2246-47a1-961a-55495217b582 : 1.64 %
51b0090e-e0c7-4b26-a38a-af92e24d3ad3 : 1.64 %
de0e4d67-db8f-4e0c-9876-d535da9d7004 : 1.64 %
b87aeb33-7dd7-45a5-a0b6-8edc3c5768bd : 0.82 %
5adb16cb-6b45-4c0e-a6ed-4796973e055a : 0.82 %
22bf17ab-c2f1-468d-8e27-357a3de6baae : 0.82 %
2be055b0-da1b-47ac-a5ed-626050768c69 : 0.82 %
6fbad

Fear : 16.56 %
Surprise : 7.01 %
Happiness : 15.29 %
Positivity : 8.28 %

[18] -------------------------------------------------
preprocessing_metadata_by_time() is completed!

node : 10
emotion : 57
10c215a2-a24f-4381-ab68-8e5e9eeec078 : 38.6 %
fda0171e-f5e5-3dcc-b69f-a54bf8269fca : 19.3 %
24068499-1301-3e4c-b81d-4af417c35bbc : 10.53 %
d7ecfbb0-2e61-4659-8594-e4c856c7c50d : 10.53 %
466a5cc2-7664-4916-9b94-a7a6b2fca37a : 7.02 %
8213a890-9dcd-427d-8236-52748e59745f : 3.51 %
80a39948-a3c4-43be-9273-c553b6909380 : 3.51 %
64cf9677-b0b3-3c95-a5f5-30eadd8d5564 : 3.51 %
481780a7-0762-46a6-b2f5-d6677307ac7c : 1.75 %
1815bf27-dea5-3dad-a201-a82f87cdecc1 : 1.75 %

add_emotion_category_id_to_metadata() is completed!

total : 57 emotions
Anger : 22.81 %
Sadness : 17.54 %
Fear : 21.05 %
Surprise : 7.02 %
Happiness : 28.07 %
Positivity : 3.51 %

[21] -------------------------------------------------
preprocessing_metadata_by_time() is completed!

node : 27
emotion : 242
d68c1780-f18c-4e47-bcdd-04535

preprocessing_metadata_by_time() is completed!

node : 37901
emotion : 361520
99d118f9-8179-41a4-8992-f7b7c211b251 : 0.23 %
33f31a8b-6c50-32ca-9f4b-c06b5de4b9e8 : 0.23 %
fdfc7172-b534-4aee-8127-1e5549de5df0 : 0.22 %
83421a05-a1d6-41b3-a50e-66b94027f32f : 0.2 %
589d4f90-fc93-42a4-a605-eff0c3f2aa0f : 0.17 %
63e33135-d3c3-4998-beb6-96feba277a2a : 0.15 %
e3a96f04-79be-4a64-87e1-6acad8c286c7 : 0.13 %
7958d919-460c-4863-bbec-8ead1112450a : 0.13 %
1aa0bd60-f318-4889-93e1-b0f7fbd48248 : 0.13 %
7a23d82f-7577-360c-a73c-f04412da9c2c : 0.13 %
6db17321-a8d6-4c84-ac14-6a328df89573 : 0.12 %
4a87e357-ddf0-4bec-a586-8afbf0cc18ae : 0.12 %
a2527220-4193-4831-b4d5-01f19bebd851 : 0.12 %
5c6c9980-39e6-4519-aa00-2ceb10606e0a : 0.12 %
a72fc9ef-c608-46c5-89ff-a9bddeb8d5a1 : 0.11 %
d9ff68f4-f551-46fb-9744-62d1025d61d1 : 0.11 %
a0577c14-3746-46dc-a7fd-2a56faa1564e : 0.11 %
59f822c2-8d55-4717-b1b0-e83a9cfe745d : 0.11 %
4b9e318a-aa0c-42f6-b795-5fd7201444a1 : 0.11 %
f7ca6327-43dd-4c13-929f-3f112bb2ffee : 0.1 %
4459

preprocessing_metadata_by_time() is completed!

node : 23
emotion : 222
81334bea-fea3-3b2a-9e84-c01ad2d0ddbf : 14.86 %
59159c2c-0112-3884-9fd1-278139bc7be0 : 12.16 %
7dcab993-e6c3-3ae5-98cb-7c8b798c106a : 10.81 %
db6adf50-1a01-3d5e-b895-5de18603a32e : 9.46 %
d68c1780-f18c-4e47-bcdd-04535c38bad2 : 7.21 %
6dfbc041-7c0e-415e-9eba-6ca2fbb2c4d5 : 7.21 %
cf83b3d4-3372-4b0d-b122-e6385a62e43e : 5.41 %
4ac7dde8-b78f-4e2c-8c11-571652eb51f0 : 5.41 %
ffc48f6e-4211-4b94-bf55-342457f2e10e : 4.05 %
1ac0d03d-70ea-4eaa-9092-51aef3799f02 : 3.6 %
60f874b4-0fa1-4d14-af3c-ce45755a2581 : 3.15 %
82410de3-bec4-4ff5-8da1-5e62c3f9d75b : 2.25 %
7b05b313-fcf6-4106-9c69-138b00f1c58c : 2.25 %
9f09a80d-5608-36ad-a304-f93daf99f28d : 2.25 %
813286bd-5e0f-4c14-874f-97e73bb7bb20 : 1.8 %
b106bc95-cab2-3157-95c6-f5e357d308a8 : 1.8 %
1de5478f-6255-45d0-b5cc-630eea3ed2ad : 1.8 %
59381cea-0727-3281-ab7a-a48389ff8ac8 : 1.35 %
c86b9c38-eca3-465d-8f14-c06d98401efd : 0.9 %
76267cd6-195e-4ada-8fc7-23eb4b0f2e5c : 0.9 %
0eec43c9-12

add_emotion_category_id_to_metadata() is completed!

total : 314200 emotions
Anger : 16.19 %
Sadness : 30.29 %
Fear : 14.48 %
Surprise : 10.04 %
Happiness : 19.96 %
Positivity : 9.05 %

[8] -------------------------------------------------
preprocessing_metadata_by_time() is completed!

node : 40
emotion : 460
50bac10a-e397-4c2b-927d-6ae979cbce41 : 30.65 %
377165f1-a9e9-4922-ae3e-2f75cfa83c5f : 19.78 %
8bc7efaa-781e-45ca-a62c-0979f4403c75 : 10.43 %
e398ffa0-71be-4ea7-9a53-3bb50380f5c5 : 5.65 %
a4ca66d2-0318-4563-a093-48c10485372e : 3.7 %
cd951ffc-2308-412d-b09f-1938b352c821 : 3.7 %
07f78ac8-e403-43a1-8c43-c99a888691f6 : 2.83 %
34d12faf-7440-4a64-b0dd-3de4cce6bb44 : 2.39 %
eb56282f-fe7e-3209-b18c-596accd5ded7 : 2.17 %
c540bb10-192c-40b7-bab9-c7df2bfe91e6 : 1.74 %
819aeffc-ed20-48c5-b9ff-9ab5fb5f12c1 : 1.74 %
5a1f25ed-ba57-4438-a607-b64e54300178 : 1.52 %
125507a7-32be-4eb0-b357-f9d6a3bff171 : 1.3 %
a212a3b3-042d-4b84-a3c8-143107fba246 : 1.09 %
478ba9d4-db87-47e5-9c32-0773f0660335 : 0.87 

preprocessing_metadata_by_time() is completed!

node : 34426
emotion : 257005
589d4f90-fc93-42a4-a605-eff0c3f2aa0f : 0.47 %
0954500c-ea20-4800-9dda-60da61b2b699 : 0.18 %
55048917-9a43-4751-bab4-6eb6ac1af8fd : 0.17 %
a30178eb-f380-3433-a74d-347f85133332 : 0.16 %
1a056287-c4aa-44a1-afae-961edade80b3 : 0.13 %
78508f29-c6b2-4bc4-bb3d-e1ef99f577d0 : 0.13 %
5936fd67-ea5d-435e-8cd8-faa3de5f29b2 : 0.12 %
d6c25ea6-5bcf-471d-9aba-b149ed3cc0fc : 0.12 %
58a81d3b-2601-4423-8196-4a1f657241e0 : 0.11 %
15e5c8a6-f332-4516-8608-fcb0324ea907 : 0.11 %
d994915a-276f-4d6f-a65c-3ca8865b04fc : 0.11 %
32e78c62-1176-47c7-91e6-253330bbd6da : 0.11 %
5e2c2b4a-6d89-46e6-b429-6fb140dcd926 : 0.11 %
4b9e318a-aa0c-42f6-b795-5fd7201444a1 : 0.11 %
83421a05-a1d6-41b3-a50e-66b94027f32f : 0.11 %
45f1abfe-b6bd-36fe-9c5a-f1fa8b190033 : 0.1 %
9f5a5c52-ce62-4808-b354-069a822ac646 : 0.1 %
56ef2015-ec82-4b9f-8e55-e47c0cd5def3 : 0.1 %
444d5a8c-0c78-39f0-91de-e2995463017e : 0.1 %
15b6741d-f200-45ec-9738-465410ab2a7c : 0.09 %
a6b0d5

total : 181 emotions
Anger : 13.26 %
Sadness : 24.86 %
Fear : 20.99 %
Surprise : 4.42 %
Happiness : 20.99 %
Positivity : 15.47 %

[24] -------------------------------------------------
preprocessing_metadata_by_time() is completed!

node : 15
emotion : 144
e18632ca-a361-3901-82b7-5f625f4d773c : 31.25 %
2f8bdcd3-4c33-42a2-aed3-03c4acd8372a : 11.81 %
4b8cb24a-5641-47ae-9d94-73207f8ba815 : 11.81 %
c5f73538-a8c9-4a8f-a4e1-2a3945f53004 : 9.72 %
f9175452-6be0-4f2f-9737-bda7aad1feea : 7.64 %
ac697387-dbe0-4bc0-ad1b-d0ba7b2d7492 : 6.25 %
475eb03e-54a7-410e-8d5a-1cfd0f92f31b : 5.56 %
2f307ce2-6091-44f6-be55-82817d3c4575 : 4.86 %
36ae7f3b-a1e5-49a3-a151-3c39b9ace732 : 2.78 %
44d5d3ab-5cee-484a-88c0-81d659ad5d62 : 2.78 %
d3892daf-b9b2-491d-8cae-2584b193f00c : 2.08 %
d9e9bc53-9cd1-441d-acd7-da479ef7e312 : 1.39 %
7bea78bf-e5a2-4691-87fd-8ed19df39fe9 : 0.69 %
1b74a66e-96a1-38f1-a734-f8fc95f2439d : 0.69 %
4f8ae00e-9469-3c8d-9c56-b54fa135fe29 : 0.69 %

add_emotion_category_id_to_metadata() is complete

add_emotion_category_id_to_metadata() is completed!

total : 276682 emotions
Anger : 15.8 %
Sadness : 32.52 %
Fear : 14.85 %
Surprise : 9.34 %
Happiness : 18.76 %
Positivity : 8.72 %

[8] -------------------------------------------------
preprocessing_metadata_by_time() is completed!

node : 47
emotion : 287
377165f1-a9e9-4922-ae3e-2f75cfa83c5f : 19.86 %
77eac667-eac4-4445-b9ba-e0af3d2f2c1d : 8.71 %
34d12faf-7440-4a64-b0dd-3de4cce6bb44 : 8.71 %
cd21e79f-f435-458d-8898-764a8c8afcf8 : 6.97 %
16878f9a-73f6-47e0-a30a-1c68327a078c : 6.27 %
790a4101-42dc-4d1f-af70-d3d7dbfef727 : 5.23 %
8bc7efaa-781e-45ca-a62c-0979f4403c75 : 4.53 %
eb56282f-fe7e-3209-b18c-596accd5ded7 : 3.83 %
8837b99e-b1aa-4c5d-9bdd-dd4e180e4e9b : 3.83 %
a212a3b3-042d-4b84-a3c8-143107fba246 : 3.48 %
22011af5-fc01-4662-9ad6-9b50cad3ca9a : 2.44 %
7889cb92-ea3a-41f3-939d-413b756e0a07 : 2.44 %
125507a7-32be-4eb0-b357-f9d6a3bff171 : 1.74 %
e432966f-c4e2-4467-85bc-69af0371dc91 : 1.74 %
e398ffa0-71be-4ea7-9a53-3bb50380f5c5 : 1.39 %

In [12]:
show_alltime_result()

[all] -------------------------------------------------
preprocessing_metadata_by_time() is completed!

node : 163696
emotion : 3549428
589d4f90-fc93-42a4-a605-eff0c3f2aa0f : 0.27 %
9f5a5c52-ce62-4808-b354-069a822ac646 : 0.17 %
3941fdbd-3679-46a0-92cf-6abfe26c8bae : 0.12 %
4b9e318a-aa0c-42f6-b795-5fd7201444a1 : 0.12 %
a30178eb-f380-3433-a74d-347f85133332 : 0.1 %
bfb2d368-74cb-41eb-99f4-605b93ad252b : 0.1 %
83421a05-a1d6-41b3-a50e-66b94027f32f : 0.1 %
0954500c-ea20-4800-9dda-60da61b2b699 : 0.09 %
33f31a8b-6c50-32ca-9f4b-c06b5de4b9e8 : 0.08 %
d6c25ea6-5bcf-471d-9aba-b149ed3cc0fc : 0.08 %
db35fb2e-98bb-4753-be1c-dfae5f64f9e1 : 0.07 %
6db17321-a8d6-4c84-ac14-6a328df89573 : 0.07 %
56ef2015-ec82-4b9f-8e55-e47c0cd5def3 : 0.07 %
195db094-0bf8-4b8a-8cda-2a0b7323723e : 0.07 %
4a87e357-ddf0-4bec-a586-8afbf0cc18ae : 0.07 %
5c6c9980-39e6-4519-aa00-2ceb10606e0a : 0.07 %
7a23d82f-7577-360c-a73c-f04412da9c2c : 0.07 %
a6ec8ecd-a377-4c36-8949-f8dcc846d0b8 : 0.06 %
174d8703-e501-4c1c-88eb-25ba49a8b4b1 : 

total : 3823 emotions
Anger : 17.21 %
Sadness : 24.54 %
Fear : 14.57 %
Surprise : 7.93 %
Happiness : 28.35 %
Positivity : 7.4 %

[11] -------------------------------------------------
preprocessing_metadata_by_time() is completed!

node : 46
emotion : 977
2295f6b6-438a-47a8-99ae-7809020448a2 : 26.2 %
bd969f27-1fef-41c8-9bd8-deb73b62615d : 14.43 %
75425bf9-0e98-44cc-9953-3ae4960ba87b : 11.57 %
5f9fd14a-a257-410c-9470-b3ac17d64a6e : 6.24 %
54a4556f-6b12-44af-8eb2-36c09d359bed : 5.12 %
6c35a5cd-2244-3761-a0b6-c3b68fcb9885 : 4.2 %
8ef78c40-c25a-4e72-9a2c-99caae3a812c : 4.2 %
01a03e43-d09a-3064-b03d-05229a061131 : 2.25 %
823244ec-181b-43a8-aa05-809447448a26 : 2.25 %
7cf718a7-30de-4830-bc13-4638bcd029cf : 2.15 %
8b8c4afc-27db-35f0-8bd9-32314fd85e10 : 1.94 %
1cc81677-cba0-4efa-9499-1e86358a8809 : 1.64 %
368527ba-4eb0-4894-a447-9acfe259fe36 : 1.64 %
81db9494-bc48-4666-9450-e16d5fb2ad68 : 1.43 %
3235d4f4-711f-4ca0-b230-a14c72ec60ab : 1.43 %
8bc8c446-5930-4211-bb0f-204266536171 : 1.33 %
1a50cc49

preprocessing_metadata_by_time() is completed!

node : 85
emotion : 2529
c5f73538-a8c9-4a8f-a4e1-2a3945f53004 : 20.56 %
ac697387-dbe0-4bc0-ad1b-d0ba7b2d7492 : 9.89 %
4b8cb24a-5641-47ae-9d94-73207f8ba815 : 9.25 %
f9175452-6be0-4f2f-9737-bda7aad1feea : 8.3 %
e18632ca-a361-3901-82b7-5f625f4d773c : 7.16 %
118522ea-a39d-4b47-b5e5-67e929303164 : 4.98 %
475eb03e-54a7-410e-8d5a-1cfd0f92f31b : 3.99 %
1e53f454-cac2-4bd0-ae4f-e2aaf8939383 : 3.01 %
8c0c998b-0d72-43e0-83af-35d45078d607 : 2.41 %
477a6037-a740-4fc5-90d2-7b185fdb1025 : 2.06 %
7bea78bf-e5a2-4691-87fd-8ed19df39fe9 : 1.82 %
44d5d3ab-5cee-484a-88c0-81d659ad5d62 : 1.74 %
2f8bdcd3-4c33-42a2-aed3-03c4acd8372a : 1.66 %
36ae7f3b-a1e5-49a3-a151-3c39b9ace732 : 1.34 %
30936ee0-fbc8-4d66-a877-0ea5aa5a86d1 : 1.27 %
f95769da-7778-4957-8e90-4e12e8c8cd15 : 1.11 %
63d5f2f6-9a8b-3cc9-b47e-3dd0ad41ca18 : 1.11 %
bd90ff42-3738-46c1-8107-c8b2b089492c : 1.07 %
cc76e548-a970-3ae5-96f1-ac74715eafcf : 1.07 %
b360f056-99dc-4e66-ba89-b27805ada2d2 : 1.07 %
4f8ae00

preprocessing_metadata_by_time() is completed!

node : 60
emotion : 1475
649e4338-5c2b-4a02-8d7a-590ba91be9c0 : 14.31 %
51501c58-2bce-4e93-a2a7-d29f59e9e96c : 9.83 %
6baab580-646b-408a-b2c4-f5b6b2d894e7 : 8.88 %
1940f295-04a9-3732-a95c-21d3975ad172 : 6.37 %
176d2d1d-b21a-4d66-8a23-37b9a948ee38 : 4.41 %
63619d53-499a-4438-a779-ba29e8e54603 : 4.14 %
bc7f5509-63bd-4b24-af03-c547d90a70eb : 4.14 %
e6a07d17-6502-4d2a-a5f9-401a897dd594 : 3.46 %
a9864d59-646a-44ec-ae69-59b73636aa3e : 3.39 %
4cd29afc-21a0-470e-b93c-9dfd265d6a86 : 3.25 %
bc4bdda4-45a9-4d28-8a13-4ae39267f70f : 3.25 %
f58c7090-2f69-3d13-8eb9-530384e22848 : 3.25 %
42114c7d-fad8-40a2-b138-c5dbd9c0e87c : 2.24 %
a291139e-3a45-49e8-9f68-3eb1c5e88190 : 1.9 %
a6c25600-e4a1-4e69-9df8-af27c0b8bb49 : 1.76 %
7c939782-6ba3-4d35-ae98-53d661900f25 : 1.69 %
f5f57ca8-af65-458a-ad1a-3899e095dce9 : 1.69 %
97787bbf-0d68-47d4-9eae-b17e43d2ef45 : 1.69 %
82f2fcfb-5547-4b41-ac44-58cc999e250c : 1.56 %
36d264ad-217a-4039-bf8e-d931de38e659 : 1.42 %
cf95053

# Don't use

In [ ]:
# コミュニティの数を取得
# 動かない
def get_community_num(community):
    return len(community)

# 各コミュニティのノード数を取得
# 動かない
def get_node_num_in_community(community):
    re = {}
    for community_id, node_list in community.items():
        re[community_id] = len(node_list)
        
    return re
        
# あるノードを持つエッジを全て取得
def get_edgelist_of_one_node(node_id, edgelist):
    edge_list_of_one_node = set()
    for edge in edgelist:
        if edge[0] == node_id or edge[1] == node_id:
            edge_list_of_one_node.add(edge)
            
    return edge_list_of_one_node

# あるコミュニティに属するノードが持つエッジをすべて取得
def get_edgelist_of_one_community(community_id, community_dic, edgelist):
    edgelist_of_one_community = set()
    
    for node_id in community_dic[community_id]:
        tmp = get_edgelist_of_one_node(node_id, edgelist)
        edgelist_of_one_community |= tmp
        
    return edgelist_of_one_community

# コミュニティのエッジリストの出力先ファイル名を取得
def get_community_edgelist_filename(community_id):
    return "../vent_data/community_edgelist{}.txt".format(community_id)

# main-あるコミュニティに属するノードが持つエッジをすべて取得してファイルに出力
def create_community_edgelist():
    print('create_modified_edgelist() is started!')

    # どのコミュニティをコミュニティを使うかは手動で決める
    community_id = input("imput a communityId")
    community_edgelist_file = get_community_edgelist_filename(community_id)
    
    # ファイルからコミュニティデータを取得
    community = get_community_from_file(community_file)
    print('get_community_from_file() is completed!')
    
    # コミュニティの検証
    #print('There are {} communities.'.format(community))
    #print(get_node_num_in_community)
    
    # ファイルからエッジリスト(加工済み)を取得
    edgelist = get_edgelist_from_file(edgelist_file_to)
    print('get_edgelist_from_file() is completed!')
    
    # あるコミュニティのエッジリストを取得
    community_edgelist = get_edgelist_of_one_community(community_id, community, edgelist)
    print('get_edgelist_of_one_community() is completed!')
    
    # 1オリジンにしておく
    community_edgelist = edgelist_to_oneorigin(community_edgelist)
    print('edgelist_to_oneorigin is completed!')
    
    # コミュニティのエッジリストをファイルに出力
    edgelist_to_file(community_edgelist, community_edgelist_file)
    print('edgelist_to_file() is completed!')
    print('All completed! {} edges are created'.format(len(community_edgelist)))